**Tutorial 2**

This tutorial is based on using  ML package Weka for machine learning. Weka is a famous machine learning software and a set of libraries that one can use within a programming language. Weka was created at the University of Waikato, New Zealnd (https://www.cs.waikato.ac.nz/ml/weka/). It is accompanied with a text book of data mining taught in schools around the world (https://www.cs.waikato.ac.nz/ml/weka/book.html). The advantage of using Weka's Python package is that the implementation of algorithms is complete, comprehsive and easy to use. Let's see below.


First install Weka's Python package.

In [ ]:
! pip install python-weka-wrapper3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.4 MB 5.3 MB/s 
     |████████████████████████████████| 1.3 MB 56.3 MB/s 
  Created wheel for python-weka-wrapper3: filename=python_weka_wrapper3-0.2.12-py3-none-any.whl size=12998753 sha256=f0a3fcff25900a70680dfec8ad877712b1ae6cd88b13410cbfea3ad212cb9b2d
  Stored in directory: /root/.cache/pip/wheels/4a/81/95/9aff7c799b9e517867cf8758a0c3dc23d923df9d4703955ce6
  Created wheel for python-javabridge: filename=python_javabridge-4.0.3-cp38-cp38-linux_x86_64.whl size=1693750 sha256=0cf6930699a946c1cff7f92a2e305b864a420f4555aa909771ebc63e39aebf28
  Stored in directory: /root/.cache/pip/wheels/ce/f3/d8/389d390e98d568ea1dbcbd36398e73360038c289d29e78c462
Successfully built python-weka-wrapper3 python-javabridge


Weka was built on Java, and below we shall be setting Java and launching it in Python environment. Don't worry about understanding this code. 

In [ ]:
import os
import sys
sys.path
sys.path.append("/usr/lib/jvm/java-11-openjdk-amd64/bin/")
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"



In [ ]:

import weka.core.jvm as jvm
jvm.start()

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['/usr/local/lib/python3.8/dist-packages/javabridge/jars/rhino-1.7R4.jar', '/usr/local/lib/python3.8/dist-packages/javabridge/jars/runnablequeue.jar', '/usr/local/lib/python3.8/dist-packages/javabridge/jars/cpython.jar', '/usr/local/lib/python3.8/dist-packages/weka/lib/weka.jar', '/usr/local/lib/python3.8/dist-packages/weka/lib/python-weka-wrapper.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


We shall now upload a dataset file. Weka works with arff format easily, it can load CSV too. We shall upload .arff file because I have defined the correct data types of variables (cagtegorical or numerical) in it already.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Outlier_Removed_german_credit_2022_12_01.arff to Outlier_Removed_german_credit_2022_12_01.arff


Let's load our dataset into memory. It will be loaded using the following code. Dataset file that I have uplaoded is german_credit.arff. Note this loaded data in moemeory is not a Pandas' data frame.

In [ ]:
from weka.core.converters import Loader
from weka.core.classes import Random
from weka.classifiers import Classifier, Evaluation

In [ ]:
loader = Loader(classname="weka.core.converters.ArffLoader")
data_file = 'Outlier_Removed_german_credit_2022_12_01.arff'
#data_file="churn.arff"
#data_file="bank.arff"
data = loader.load_file(data_file)

print('Data set size: ', data.num_instances)

Data set size:  975


In [ ]:
#Let's look at the attributes and their types
# We have two data types here: categorical and numeric.
for i in range(data.num_attributes):
  print ("index ",i)
  print(data.attribute(i))

index  0
@attribute Creditability {0,1}
index  1
@attribute 'Account Balance' {1,2,3,4}
index  2
@attribute 'Duration of Credit (month)' numeric
index  3
@attribute 'Payment Status of Previous Credit' {0,1,2,3,4}
index  4
@attribute Purpose {0,1,2,3,4,5,6,8,9,10}
index  5
@attribute 'Credit Amount' numeric
index  6
@attribute 'Value Savings/Stocks' {1,2,3,4,5}
index  7
@attribute 'Length of current employment' {1,2,3,4,5}
index  8
@attribute 'Instalment per cent' numeric
index  9
@attribute 'Sex & Marital Status' {1,2,3,4}
index  10
@attribute Guarantors {1,2,3}
index  11
@attribute 'Duration in Current address' {1,2,3,4}
index  12
@attribute 'Most valuable available asset' {1,2,3,4}
index  13
@attribute 'Age (years)' numeric
index  14
@attribute 'Concurrent Credits' {1,2,3}
index  15
@attribute 'Type of apartment' {1,2,3}
index  16
@attribute 'No of Credits at this Bank' numeric
index  17
@attribute Occupation {1,2,3,4}
index  18
@attribute 'No of dependents' numeric
index  19
@attrib

Index of class attribute in our data is 0--creditability. It can be observed above. I am setting up class attribute here.

In [ ]:

# index of class atrribute is 0 (Creditability) for German credit card
# index of class attribute is 20(Churn) for Churn data set 
# index of class attribute is 16(y) for bank data set
# Again, you can see all the index numbers for attributes by running the previous cell
class_idx=0
print('Will be classifying on: ', data.attribute(class_idx))
data.class_index = class_idx


Will be classifying on:  @attribute Creditability {0,1}


Time to split dataset into train and test set.

In [ ]:
# Splitting 70% for training and 30% for testing using a seed of 1 for random number generator
train, test = data.train_test_split(70.0, Random(1))

We are now going to train a decision tree. This decision tree is C4.5 decision tree and it's name in Weka is J48. Good thing about this decision tree is that it is the exact implementation of the C4.5 decision tree as in theory and as we studied. C4.5 decision tree algorithm can handle numeric and categorical attributes by itself. So there is no need to convert categorical features(or variables) to numeric features by using on-hot-encoding.

In [ ]:
# We are generating a pruned C4.5 decision tree, with a confidence factor used for pruning of 0.25.
# You can change it to different threshold values to change the size of the tree.
cls = Classifier(classname="weka.classifiers.trees.J48", options=["-C", "0.25"])
cls.build_classifier(train)
# See the tree below. 
print(cls)

J48 pruned tree
------------------

Account Balance = 1
|   Duration of Credit (month) <= 8: 1 (13.0/1.0)
|   Duration of Credit (month) > 8
|   |   Payment Status of Previous Credit = 0: 0 (7.0/1.0)
|   |   Payment Status of Previous Credit = 1
|   |   |   Age (years) <= 23: 1 (3.0)
|   |   |   Age (years) > 23: 0 (10.0)
|   |   Payment Status of Previous Credit = 2
|   |   |   Guarantors = 1
|   |   |   |   Value Savings/Stocks = 1
|   |   |   |   |   Concurrent Credits = 1: 0 (4.0/2.0)
|   |   |   |   |   Concurrent Credits = 2: 1 (4.0/1.0)
|   |   |   |   |   Concurrent Credits = 3
|   |   |   |   |   |   Occupation = 1: 0 (1.0)
|   |   |   |   |   |   Occupation = 2
|   |   |   |   |   |   |   No of dependents <= 1
|   |   |   |   |   |   |   |   Type of apartment = 1: 1 (2.0)
|   |   |   |   |   |   |   |   Type of apartment = 2: 0 (9.0/3.0)
|   |   |   |   |   |   |   |   Type of apartment = 3: 0 (0.0)
|   |   |   |   |   |   |   No of dependents > 1: 0 (3.0)
|   |   |   |   |  

In the above tree, these values ": 1 (8.0/2.0)" means the class at the leaf is 1, total training records during evlaution on the training set after building the tree reached here are 8 but only 2 of them were incorrectly predicted.

In [ ]:
#import weka.plot.graph as graph  # If pygrpahviz is installed, you can plot the graph of tree too but it may not work
#graph.plot_dot_graph(cls.graph)

In [ ]:
# Let's evaluate it on the test set

evl = Evaluation(train)
evl.test_model(cls, test)
print(evl.summary())


Correctly Classified Instances         220               75.0853 %
Incorrectly Classified Instances        73               24.9147 %
Kappa statistic                          0.3313
Mean absolute error                      0.3484
Root mean squared error                  0.4259
Relative absolute error                 84.809  %
Root relative squared error             94.1852 %
Total Number of Instances              293     



Here "Correctly Classified Instances"   means accuracy, and "Total Number of Instances" means total records in the test set. Ignore everything else as we have not studied them. 

In [ ]:
# Here are all the metrics
#print ("Class Index ", class_idx)
print("Classes at different positions are ",data.attribute(class_idx))

print("confusion Matrix")
#Note that the TP here will be for the class at the first position printed by the previous line and TN will be for the class at second position
print(evl.confusion_matrix)

###############
# Print metrics for the first class
##############
class_position=0
print("")
print ("Evaluation from the perspective of class at position "+ str(class_position))
print("TP ",evl.true_positive_rate(class_position))
print("FP",evl.false_positive_rate(class_position))
print("Precision ",evl.precision(class_position))
print("Recall ",evl.recall(class_position))


###############
# Print metrics for the second class
##############
class_position=1
print("")
print ("Evaluation from the perspective of class at position "+ str(class_position))
print("TP ",evl.true_positive_rate(class_position))
print("FP",evl.false_positive_rate(class_position))
print("Precision ",evl.precision(class_position))
print("Recall ",evl.recall(class_position))


Classes at different positions are  @attribute Creditability {0,1}
confusion Matrix
[[ 35.  49.]
 [ 24. 185.]]

Evaluation from the perspective of class at position 0
TP  0.4166666666666667
FP 0.11483253588516747
Precision  0.5932203389830508
Recall  0.4166666666666667

Evaluation from the perspective of class at position 1
TP  0.8851674641148325
FP 0.5833333333333334
Precision  0.7905982905982906
Recall  0.8851674641148325


**Naive Bayes**

Below is the code to run Naive Bayes algorithm. It is a different version of Naive Bayes that is suited to both numeric and categorical features(atrributes or variables).
 (https://weka.sourceforge.io/doc.dev/weka/classifiers/bayes/NaiveBayes.html)

In [ ]:

nb = Classifier(classname="weka.classifiers.bayes.NaiveBayes")
nb.build_classifier(train)
#let's understand the NB model by printing it
print(nb)

Naive Bayes Classifier

                                        Class
Attribute                                   0         1
                                       (0.29)    (0.71)
Account Balance
  1                                       91.0      89.0
  2                                       64.0     105.0
  3                                       10.0      37.0
  4                                       37.0     257.0
  [total]                                202.0     488.0

Duration of Credit (month)
  mean                                 23.8801   19.0948
  std. dev.                            12.7811   10.8691
  weight sum                               198       484
  precision                             1.8667    1.8667

Payment Status of Previous Credit
  0                                       16.0       9.0
  1                                       19.0      16.0
  2                                      113.0     245.0
  3                                       21.0      39.

In [ ]:
# Time for evaluation on the test set
evl_nb = Evaluation(train)
evl_nb.test_model(nb, test)
print(evl_nb.summary())


Correctly Classified Instances         223               76.1092 %
Incorrectly Classified Instances        70               23.8908 %
Kappa statistic                          0.4281
Mean absolute error                      0.3021
Root mean squared error                  0.4119
Relative absolute error                 73.5284 %
Root relative squared error             91.0917 %
Total Number of Instances              293     



In [ ]:
#Here are all the metrics for Naive Bayes

print("Classes at different positions are ",data.attribute(class_idx))

print("confusion Matrix")
#Note that the TP here will be for the class at the first position printed by the previous line
print(evl_nb.confusion_matrix)

###############
# Print metrics for the first class
##############
class_position=0
print("")
print ("Evaluation from the perspective of class at position "+ str(class_position))
print("TP ",evl_nb.true_positive_rate(class_position))
print("FP",evl_nb.false_positive_rate(class_position))
print("Precision ",evl_nb.precision(class_position))
print("Recall ",evl_nb.recall(class_position))


###############
# Print metrics for the second class
##############
class_position=1
print("")
print ("Evaluation from the perspective of class at position "+ str(class_position))
print("TP ",evl_nb.true_positive_rate(class_position))
print("FP",evl_nb.false_positive_rate(class_position))
print("Precision ",evl_nb.precision(class_position))
print("Recall ",evl_nb.recall(class_position))


Classes at different positions are  @attribute Creditability {0,1}
confusion Matrix
[[ 52.  32.]
 [ 38. 171.]]

Evaluation from the perspective of class at position 0
TP  0.6190476190476191
FP 0.18181818181818182
Precision  0.5777777777777777
Recall  0.6190476190476191

Evaluation from the perspective of class at position 1
TP  0.8181818181818182
FP 0.38095238095238093
Precision  0.8423645320197044
Recall  0.8181818181818182


**Appendix**

Using the following code  you can find out the best attribute by using the BestFIRst algorithm in Weka. Again it is not necessary to understand the whole code below but if you wanna learn more about BesrFirst and CfsSubsetEval, you can go here https://weka.sourceforge.io/doc.dev/weka/attributeSelection/package-summary.html. You can also replace them with options available on the above site.


In [ ]:
from weka.attribute_selection import ASSearch, ASEvaluation, AttributeSelection
search = ASSearch(classname="weka.attributeSelection.BestFirst", options=["-D", "1", "-N", "5"])
evaluator = ASEvaluation(classname="weka.attributeSelection.CfsSubsetEval", options=["-P", "1", "-E", "1"])
attsel = AttributeSelection()
attsel.search(search)
attsel.evaluator(evaluator)
attsel.select_attributes(data)

print("# attributes: " + str(attsel.number_attributes_selected))
print("attributes: " + str(attsel.selected_attributes))
print("result string:\n" + attsel.results_string)

# attributes: 4
attributes: [ 1  2  3 15  0]
result string:


=== Attribute Selection on all input data ===

Search Method:
	Best first.
	Start set: no attributes
	Search direction: forward
	Stale search after 5 node expansions
	Total number of subsets evaluated: 134
	Merit of best subset found:    0.073

Attribute Subset Evaluator (supervised, Class (nominal): 1 Creditability):
	CFS Subset Evaluator
	Including locally predictive attributes

Selected attributes: 2,3,4,16 : 4
                     Account Balance
                     Duration of Credit (month)
                     Payment Status of Previous Credit
                     Type of apartment



Weka's best first search method resulted into above attributes selection. Let's create a new copy of dataset with those attributes only

In [ ]:
# As you see above, we only attributes 2,3 and 4 are important as judged by Weka for German Credit card data set. So we are going to load 
# data again and remove all the attributes from 5-21. Atrribute at index 1 is the class atrribute, so we'll keep that too
from weka.filters import Filter

data2 = loader.load_file(data_file)
# Filtering method 1
remove = Filter(classname="weka.filters.unsupervised.attribute.Remove", options=["-R", "5-21"])
remove.inputformat(data2)
filtered_data = remove.filter(data2)

print(filtered_data.subset(row_range="1-10"))

@relation 'german_credit-weka.filters.unsupervised.attribute.InterquartileRange-Rfirst-last-O3.0-E6.0-P-weka.filters.unsupervised.instance.RemoveWithValues-S0.0-C24-Llast-weka.filters.unsupervised.instance.RemoveWithValues-S0.0-C22-Llast-weka.filters.unsupervised.attribute.Remove-R5-21-weka.filters.unsupervised.instance.RemoveRange-V-R1-10-weka.filters.MultiFilter-Fweka.filters.unsupervised.instance.RemoveRange -V -R 1-10-S1'

@attribute Creditability {0,1}
@attribute 'Account Balance' {1,2,3,4}
@attribute 'Duration of Credit (month)' numeric
@attribute 'Payment Status of Previous Credit' {0,1,2,3,4}

@data
1,1,18,4
1,1,9,4
1,2,12,2
1,1,12,4
1,1,12,4
1,1,10,4
1,1,8,4
1,1,6,4
1,4,18,4
1,2,24,2


In [ ]:
#Filtering method 2
#Another way of filtering columns usingthe following code. Here we are keeping only features 1-4 and 7.
filtered_data=data2.subset(col_range='1-4,7')


Now you can remove the above filtered data set as an input data set in the code examples shown above and repeat the experiments.

More examples on the use of different functionalities of Weka's Python package are here for curious readers:
http://fracpete.github.io/python-weka-wrapper3/examples.html

In [ ]:
#If you are done stop the JVM (Java Virtual Machine)
jvm.stop()

It turns out that Weka's python package is easier and comprehensive than other Python packages.



```
For CIND 119 course at Ryerson
  by Syed Shariyar Murtaza,Ph.D.
```

